In [1]:
import pymongo
from pymongo import MongoClient
client = MongoClient()

In [2]:
db = client.nobel_prizes

In [3]:
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'nobel_prizes')

In [4]:
collection = db.nobel

In [5]:
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'nobel_prizes'), 'nobel')

In [5]:
collection.find_one()

{'_id': ObjectId('5e65af567fa7712662f38f99'),
 'year': '2019',
 'category': 'literature',
 'laureates': [{'id': '980',
   'firstname': 'Peter',
   'surname': 'Handke',
   'motivation': '"for an influential work that with linguistic ingenuity has explored the periphery and the specificity of human experience"',
   'share': '1'}]}

In [7]:
import pprint
for post in collection.find({'year':'2015'}):
    pprint.pprint(post)

{'_id': ObjectId('5e65af567fa7712662f38fac'),
 'category': 'economics',
 'laureates': [{'firstname': 'Angus',
                'id': '926',
                'motivation': '"for his analysis of consumption, poverty, and '
                              'welfare"',
                'share': '1',
                'surname': 'Deaton'}],
 'year': '2015'}
{'_id': ObjectId('5e65af567fa7712662f38fae'),
 'category': 'literature',
 'laureates': [{'firstname': 'Svetlana',
                'id': '924',
                'motivation': '"for her polyphonic writings, a monument to '
                              'suffering and courage in our time"',
                'share': '1',
                'surname': 'Alexievich'}],
 'year': '2015'}
{'_id': ObjectId('5e65af567fa7712662f38faf'),
 'category': 'peace',
 'laureates': [{'firstname': 'National Dialogue Quartet ',
                'id': '925',
                'motivation': '"for its decisive contribution to the building '
                              'of a plu

In [8]:
collection.count_documents({'year':'2019'})

6

In [8]:
for post in collection.find({'laureates.firstname':'Marie'}, {'category': 1, 'year':1,  'laureates.firstname': 1, 'laureates.surname': 1}):
    pprint.pprint(post)

{'_id': ObjectId('5e65af567fa7712662f391e8'),
 'category': 'chemistry',
 'laureates': [{'firstname': 'Marie', 'surname': 'Curie'}],
 'year': '1911'}
{'_id': ObjectId('5e65af567fa7712662f39210'),
 'category': 'physics',
 'laureates': [{'firstname': 'Henri', 'surname': 'Becquerel'},
               {'firstname': 'Pierre', 'surname': 'Curie'},
               {'firstname': 'Marie', 'surname': 'Curie'}],
 'year': '1903'}


In [10]:
for post in collection.find({'laureates.firstname':'Marie'}, {'laureates.motivation':0, 'laureates.share': 0}):
    pprint.pprint(post)

{'_id': ObjectId('5e65af567fa7712662f391e8'),
 'category': 'chemistry',
 'laureates': [{'firstname': 'Marie', 'id': '6', 'surname': 'Curie'}],
 'year': '1911'}
{'_id': ObjectId('5e65af567fa7712662f39210'),
 'category': 'physics',
 'laureates': [{'firstname': 'Henri', 'id': '4', 'surname': 'Becquerel'},
               {'firstname': 'Pierre', 'id': '5', 'surname': 'Curie'},
               {'firstname': 'Marie', 'id': '6', 'surname': 'Curie'}],
 'year': '1903'}


In [9]:
for post in collection.find({'laureates.firstname':'Marie'}, {'year': 1, 'laureates.firstname': 1, 'laureates.surname': 1}):
    pprint.pprint(post)

{'_id': ObjectId('5e65af567fa7712662f391e8'),
 'laureates': [{'firstname': 'Marie', 'surname': 'Curie'}],
 'year': '1911'}
{'_id': ObjectId('5e65af567fa7712662f39210'),
 'laureates': [{'firstname': 'Henri', 'surname': 'Becquerel'},
               {'firstname': 'Pierre', 'surname': 'Curie'},
               {'firstname': 'Marie', 'surname': 'Curie'}],
 'year': '1903'}


In [10]:
for post in collection.aggregate([{ '$group': { '_id': '$category', 'num_doc': {'$sum': 1} } } ]):
    pprint.pprint(post)

{'_id': 'medicine', 'num_doc': 119}
{'_id': 'economics', 'num_doc': 51}
{'_id': 'peace', 'num_doc': 119}
{'_id': 'chemistry', 'num_doc': 119}
{'_id': 'literature', 'num_doc': 119}
{'_id': 'physics', 'num_doc': 119}


In [11]:
for post in collection.aggregate([ 
            { '$group': { '_id': '$category',  'awards': {'$sum': 
                                                       {'$size': 
                                                        {"$cond": [ 
                                                            { "$isArray": "$laureates" }, 
                                                            "$laureates", 
                                                            []
                                                            ]
                                                        }
                                                       }
                                                      }
                        }
            } ] ):
    pprint.pprint(post)

{'_id': 'medicine', 'awards': 219}
{'_id': 'economics', 'awards': 84}
{'_id': 'peace', 'awards': 134}
{'_id': 'literature', 'awards': 116}
{'_id': 'chemistry', 'awards': 184}
{'_id': 'physics', 'awards': 213}


In [12]:
from bson.code import Code
mapper = Code("""
        function () {
              emit('a', 1);
        }
        """)
reducer = Code("""
              function (key, values) {
                 var total = 0;
                 for (var i = 0; i < values.length; i++) {
                 total += values[i];
                 }
               return total;
      }
        """)
result = collection.map_reduce(mapper, reducer, "myresults")
for doc in result.find():
    pprint.pprint(doc)

{'_id': 'a', 'value': 646.0}


In [13]:
mapper = Code("""
        function () {
              emit(this.category, 1);
        }
        """)
reducer = Code("""
              function (key, values) {
                 var total = 0;
                 for (var i = 0; i < values.length; i++) {
                 total += values[i];
                 }
               return total;
      }
        """)
result = collection.map_reduce(mapper, reducer, "myresults")
for doc in result.find():
    pprint.pprint(doc)

{'_id': 'chemistry', 'value': 119.0}
{'_id': 'economics', 'value': 51.0}
{'_id': 'literature', 'value': 119.0}
{'_id': 'medicine', 'value': 119.0}
{'_id': 'peace', 'value': 119.0}
{'_id': 'physics', 'value': 119.0}


In [13]:
mapper = Code("""
        function () {
              var c = this.category;
              if (Array.isArray(this.laureates)) {
                this.laureates.forEach (function (z) {
                  emit (c, 1);
              });
              }
        }
        """)
reducer = Code("""
              function (key, values) {
                 var total = 0;
                 for (var i = 0; i < values.length; i++) {
                 total += values[i];
                 }
               return total;
      }
        """)
result = collection.map_reduce(mapper, reducer, "myresults")
for doc in result.find():
    pprint.pprint(doc)

{'_id': 'chemistry', 'value': 184.0}
{'_id': 'economics', 'value': 84.0}
{'_id': 'literature', 'value': 116.0}
{'_id': 'medicine', 'value': 219.0}
{'_id': 'peace', 'value': 134.0}
{'_id': 'physics', 'value': 213.0}


In [14]:
mapper = Code("""
        function () {
              if (Array.isArray(this.laureates)) {
                this.laureates.forEach (function (z) {
                  var fn = z.firstname;
                  emit (fn, 1);
              });
              }
        }
        """)
reducer = Code("""
              function (key, values) {
                 var total = 0;
                 for (var i = 0; i < values.length; i++) {
                 total += values[i];
                 }
               return total;
      }
        """)
result = collection.map_reduce(mapper, reducer, "myresults")
for doc in result.find():
    pprint.pprint(doc)

{'_id': 'A. Michael', 'value': 1.0}
{'_id': 'Aage N.', 'value': 1.0}
{'_id': 'Aaron', 'value': 2.0}
{'_id': 'Abdus', 'value': 1.0}
{'_id': 'Abhijit', 'value': 1.0}
{'_id': 'Abiy', 'value': 1.0}
{'_id': 'Ada E.', 'value': 1.0}
{'_id': 'Adam G.', 'value': 1.0}
{'_id': 'Adolf', 'value': 3.0}
{'_id': 'Adolfo', 'value': 1.0}
{'_id': 'Ahmed', 'value': 1.0}
{'_id': 'Akira', 'value': 2.0}
{'_id': 'Al', 'value': 1.0}
{'_id': 'Alan', 'value': 3.0}
{'_id': 'Albert', 'value': 8.0}
{'_id': 'Albert A.', 'value': 1.0}
{'_id': 'Albrecht', 'value': 1.0}
{'_id': 'Aleksandr M.', 'value': 1.0}
{'_id': 'Alexandr', 'value': 1.0}
{'_id': 'Alexei', 'value': 1.0}
{'_id': 'Alexis', 'value': 1.0}
{'_id': 'Alfonso', 'value': 1.0}
{'_id': 'Alfred', 'value': 3.0}
{'_id': 'Alfred D.', 'value': 1.0}
{'_id': 'Alfred G.', 'value': 1.0}
{'_id': 'Alice', 'value': 1.0}
{'_id': 'Allan M.', 'value': 1.0}
{'_id': 'Allvar', 'value': 1.0}
{'_id': 'Alphonse', 'value': 1.0}
{'_id': 'Alva', 'value': 1.0}
{'_id': 'Alvin E.', 'valu

In [15]:
mapper = Code("""
        function () {
              if (Array.isArray(this.laureates)) {
                this.laureates.forEach (function (z) {
                  var fn = z.firstname;
                  emit (fn, 1);
              });
              }
        }
        """)
reducer = Code("""
              function (key, values) {
                 var total = 0;
                 for (var i = 0; i < values.length; i++) {
                 total += values[i];
                 }
               return total;
      }
        """)
result = collection.map_reduce(mapper, reducer, "myresults")
for doc in result.find().sort([('value',-1)]):
    pprint.pprint(doc)

{'_id': 'John', 'value': 12.0}
{'_id': 'Paul', 'value': 9.0}
{'_id': 'Albert', 'value': 8.0}
{'_id': 'Peter', 'value': 8.0}
{'_id': 'James', 'value': 7.0}
{'_id': 'Otto', 'value': 7.0}
{'_id': 'Robert', 'value': 7.0}
{'_id': 'William', 'value': 7.0}
{'_id': 'Richard', 'value': 6.0}
{'_id': 'Carl', 'value': 5.0}
{'_id': 'Hans', 'value': 5.0}
{'_id': 'Max', 'value': 5.0}
{'_id': 'Arthur', 'value': 4.0}
{'_id': 'Frederick', 'value': 4.0}
{'_id': 'George', 'value': 4.0}
{'_id': 'Henri', 'value': 4.0}
{'_id': 'John C.', 'value': 4.0}
{'_id': 'Karl', 'value': 4.0}
{'_id': 'Martin', 'value': 4.0}
{'_id': 'Michael', 'value': 4.0}
{'_id': 'Adolf', 'value': 3.0}
{'_id': 'Alan', 'value': 3.0}
{'_id': 'Alfred', 'value': 3.0}
{'_id': 'Christian', 'value': 3.0}
{'_id': 'Daniel', 'value': 3.0}
{'_id': 'David J.', 'value': 3.0}
{'_id': 'Eric', 'value': 3.0}
{'_id': 'Ernest', 'value': 3.0}
{'_id': 'François', 'value': 3.0}
{'_id': 'Fritz', 'value': 3.0}
{'_id': 'Frédéric', 'value': 3.0}
{'_id': 'Gerhar

In [16]:
mapper = Code("""
        function () {
              var c = this.category;
              if (Array.isArray(this.laureates)) {
                this.laureates.forEach (function (z) {
                  var fn = z.firstname;
                  emit (fn, c);
              });
              }
        }
        """)
reducer = Code("""
              function (key, values) {
               return values.toString();
      }
        """)
result = collection.map_reduce(mapper, reducer, "myresults")
for doc in result.find().sort([('value',-1)]):
    pprint.pprint(doc)

{'_id': 'David J.', 'value': 'physics,physics,physics'}
{'_id': 'Wolfgang', 'value': 'physics,physics,physics'}
{'_id': 'Louis', 'value': 'physics,physics,peace'}
{'_id': 'Jack', 'value': 'physics,physics'}
{'_id': 'Steven', 'value': 'physics,physics'}
{'_id': 'Georges', 'value': 'physics,peace'}
{'_id': 'Albert',
 'value': 'physics,medicine,peace,literature,peace,medicine,physics,peace'}
{'_id': 'Carl', 'value': 'physics,medicine,peace,chemistry,literature'}
{'_id': 'Arthur', 'value': 'physics,medicine,peace,chemistry'}
{'_id': 'Hans', 'value': 'physics,medicine,medicine,chemistry,chemistry'}
{'_id': 'François', 'value': 'physics,medicine,literature'}
{'_id': 'George E.', 'value': 'physics,medicine'}
{'_id': 'J. Michael', 'value': 'physics,medicine'}
{'_id': 'Heinrich', 'value': 'physics,literature,chemistry'}
{'_id': 'Claude', 'value': 'physics,literature'}
{'_id': 'Eugene', 'value': 'physics,literature'}
{'_id': 'Rudolf', 'value': 'physics,literature'}
{'_id': 'Saul', 'value': 'phys